In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
from langchain_groq import ChatGroq

In [3]:
import csv

csv_file_path = "SystemReq_EPS.csv"

def load_csv(file_path):
    requirements = []
    with open(file_path, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            requirements.append(row)
    return requirements

In [4]:
requirements_doc = load_csv(csv_file_path)

from langchain_core.documents import Document

documents = [(Document(page_content=str( {
            "Primary Text": item['Primary Text'],
            "Verification_Criteria": item['Verification_Criteria'],
        }), metadata={"Requirement ID": item["id"]})) for item in requirements_doc]

In [5]:
import uuid
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

ids = [str(uuid.uuid4()) for _ in range(len(documents))] 

unique_collection_name = f"collection_{uuid.uuid4()}"

client = Chroma(collection_name=unique_collection_name, embedding_function=embeddings_model)

# Add documents
vector_doc = client.add_documents(documents=documents, ids = ids)

In [596]:
ORCHESTRATOR_PROMPT = """ You are provided with requirements of Automotive Electronic Control Unit(ECU). 
Analyze the requirement and understand the context of the it based on documents provided. You should understand the requirement completely, if you have any doubt please ask for clarifications
        requirement: {requirement}
        documents: {documents}

Return your response in this format:

<analysis>
Explain your understanding of the given requirement in detail. Break down it in detail. Don't include any "PREAMBLE"

Based on the analysis, explain how the respective requirement can be fully validated / tested with "NO PREAMBLE". 
</analysis>

"""

In [592]:
WORKER_PROMPT = """ You are a test case generation assistant who will create test cases for given Test Scenarios. 
Both requirement context and test sceanrios is given to you as input, your task is to understand the requirement context and generate the test cases for each test sceanrios.

Requirements context and test scenarios:{context_requirement} 

Return your response in below form for each test case containing below entries, without any additional information (No Preamble):

<test_cases>
Test Case ID : ID for test case (should be whole number)
Objective : Objective of test case
Test Design Technique : Test design technique used for this test
Pre-conditions : What are the preconditions before performing the test
Steps to Execute : Detailed test steps
Expected Results : Expected outcome for respective Execution step 
</test_cases>

Note: Test sceanrios is given to you after ANALYSIS. 
"""

In [414]:
import re
from langchain.memory import ConversationBufferWindowMemory 
from langchain.chains import ConversationChain

def extract_xml(text: str, tag: str) -> str:
    """
    Extracts the content of the specified XML tag from the given text. Used for parsing structured responses 

    Args:
        text (str): The text containing the XML.
        tag (str): The XML tag to extract content from.

    Returns:
        str: The content of the specified XML tag, or an empty string if the tag is not found.
    """
    match = re.search(f'<{tag}>(.*?)</{tag}>', text, re.DOTALL)
    return match.group(1) if match else ""

# retrieving the relevant documents from vector store
def retrieve_documents(query_text, collection, top_k=4):
    """Fetching relevant document from vector store """
        
    documents=[]
    retriever = collection.as_retriever()
    retrieve_doc = retriever.invoke(query_text)
    
    for item in retrieve_doc:
         documents.append({
                "content": item.page_content,
                "source": item.metadata,
            })  
    
    return documents

#cleanising the retrieved vector store document
def clean_text(relevant_document):
    content_list = []
    for item in relevant_document:
        # Clean the 'content' field
        content = item['content']
        content = content.replace("'", '"')
        content = re.sub(r'[\'\-•]""', "", content)  # Remove specified characters
        content = re.sub(r'\s{2,}', ' ', content).strip()  # Normalize whitespace and trim
        item['content'] = content

        # Replace data in 'Verification_Criteria' field if it exists
        #for var in item:
        # criteria = item['Verification_Criteria']  # Update cleaned criteria in the item
        
        # combined_content = f"Primary Text: {content}\nVerification Criteria: {criteria}\n\n"
        content_list.append(content)
        cleanised_document = ''.join(content_list)
    return cleanised_document

def llm_call_orches(prompt: str, system_prompt: str = "", model="llama3-70b-8192") -> str:
    """
    Calls the model with the given prompt and returns the response.

    Args:
        prompt (str): The user prompt to send to the model.
        system_prompt (str, optional): The system prompt to send to the model. Defaults to "".
        model (str, optional): The model to use for the call. Defaults to "llama3-70b-8192".

    Returns:
        str: The response from the language model.
    """
    
    # messages = [{"role": "user", "content": prompt}]
    memory = ConversationBufferWindowMemory(k=10)
    
    llm = ChatGroq(
    model=model,
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY")
    )
    
    conversation = ConversationChain(
                    llm=llm,
                    memory=memory
    )
    response = conversation.invoke(prompt)
    # memory.save_context({"input": ""}, {"output": str(response['response'])})
    # print(memory.load_memory_variables({}))
    
    return response

In [552]:
from typing import Dict, List, Optional
from pprint import pprint
    
class Orchestrator:
    """Break down tasks and run them in parallel using worker LLMs."""
    
    def __init__(self,orchestrator_prompt: str, worker_prompt:str):
        self.orchestrator_prompt = orchestrator_prompt
        self.worker_prompt = worker_prompt
    
    # formating prompt by replacing {} variables with its actual value 
    def _format_prompt(self, template: str,  **kwargs) -> str:
        """Format a prompt template with variables."""
        try:
            return template.format(**kwargs)
        except KeyError as e:
            raise ValueError(f"Missing required prompt variable: {e}")

    def process_llm(self, requirement: str, client: object) -> Dict:
        """Orchestrate tasks using the LLM based on the given requirement.
        
        Args:
            requirement (str): The requirement to process.
        
        Returns:
            str: Output from the orchestrator.
        """
        unpack = {}
        
        # retrieving and cleansing the document
        raw_documents = retrieve_documents(requirement,client)
        documents = clean_text(raw_documents)
    
        orchestrator_input = self._format_prompt(
            self.orchestrator_prompt,
            requirement=requirement,
            documents=documents,
            **unpack
        )
        orchestrator_response = llm_call_orches(orchestrator_input)
        
        analysis = extract_xml(orchestrator_response['response'], "analysis")
        #test_scenarios = extract_xml(orchestrator_response['response'], "tests")
        
        print("\n=== ORCHESTRATOR OUTPUT ===")
        # pprint(orchestrator_response)
        print(f"\nANALYSIS:\n{analysis}")
        #print(f"\nTEST SCENARIOS:\n{test_scenarios}")

        # #Step 2: Process each task
        # worker_input = self._format_prompt(
        #         self.worker_prompt,
        #         context_requirement=analysis,
        #         test_conditions=test_scenarios,
        #         **unpack
        # )
            
        # worker_response = llm_call(worker_input)
        # pprint(worker_response)
        
        # TC_result = extract_xml(worker_response['response'], "test_cases")
            
        # print(f"\n=== WORKER RESULT ===\n{TC_result}\n")
        
        return analysis
        #return {"Analysis": analysis, "Test Scenarios": test_scenarios}

In [553]:
sub_requirement = requirements_doc[3:7]
sub_requirement

[{'id': '245103',
  'Primary Text': 'Steering Wheel Angle:  \nInput Range - ±450° (effective within ±180°)\nOutput Range - Steering Angle:8° at wheels (per 90° input), Assist Torque: 1Nm–15Nm.\nSteering Torque:\nInput Range - 0–15Nm (varies with maneuver)\nOutput Range - Assist Torque: 1Nm–15Nm, assist ratio up to 4:1 Turning Radius: 4–6 meters (parking)\nVehicle Speed:  \nInput Range - 0 km/h to 200 km/h\nOutput Range - Assist Torque: 1Nm–10Nm (varies with speed)',
  'Artifact Type': 'SYS Requirement',
  'Safety': 'ASIL B',
  'Verification_Criteria': 'Verify the ECU behaviour for all Input / Output Range values',
  'isHeading': ''},
 {'id': '245104',
  'Primary Text': 'Application Functions',
  'Artifact Type': 'Heading',
  'Safety': '',
  'Verification_Criteria': '',
  'isHeading': 'TRUE'},
 {'id': '245105',
  'Primary Text': "The Steering ECU shall process the driver's steering input by adjusting the steering angle at the wheels and providing the appropriate assistive torque based o

In [597]:
orchestrator = Orchestrator(
    orchestrator_prompt=ORCHESTRATOR_PROMPT,
    worker_prompt=WORKER_PROMPT,
)

result_analysis = []
# Iterate through the requirements document
for item in sub_requirement:
    if item['Artifact Type'] == 'SYS Requirement':
        print(f"*********** Understanding context of Requirement {item['id']} and generating Test sceanrios to validate requirement **************")

        # Process the requirement using the orchestrator
        orchestrato_response = orchestrator.process_llm(requirement=item['Primary Text'], client=client)
        result_analysis.append(orchestrato_response)
        # Prepare the test case dictionary
       #  test_scenarios = result_analysis['Test Scenarios'].split("\n\n")      
        
       #  test_cases_dict = {
       #       'Requirement ID': item['id'],
       #       'Analysis': result_analysis['Analysis'],
       #       'Test Scenarios': result_analysis['Test Scenarios']    
       #      #'Test Scenarios': list(filter(lambda x: x and x.strip(), test_scenarios))
       #  }

       # #  # Add the test case dictionary to the final result
       #  final_result.append(test_cases_dict)

*********** Understanding context of Requirement 245103 and generating Test sceanrios to validate requirement **************

=== ORCHESTRATOR OUTPUT ===

ANALYSIS:


The given requirement is for an Automotive Electronic Control Unit (ECU) that controls the steering system of a vehicle. The ECU takes input from the steering wheel angle, steering torque, and vehicle speed to provide assistive torque and adjust the steering angle to facilitate vehicle maneuvering.

The requirement can be broken down into three main components:

1. Steering Wheel Angle:
	* Input range: ±450° (effective within ±180°)
	* Output range: Steering angle: 8° at wheels (per 90° input), Assist torque: 1Nm-15Nm
	* The ECU shall adjust the steering angle based on the steering wheel input, with an assist ratio of up to 4:1.

2. Steering Torque:
	* Input range: 0-15Nm (varies with maneuver)
	* Output range: Assist torque: 1Nm-15Nm, assist ratio up to 4:1
	* The ECU shall generate assistive torque based on the steering

In [598]:
sub_analysis = result_analysis[1]

In [604]:
EVAL_PROMPT = """ You are expert in software testing and well aware testing standards. You are provided with certain test cases and its corresponding requirement context 
Your task is to evaluate:
- Whether the test cases achieve the maximum coverage for input requirement context.
- Whether the test cases are written as per standard testing guidelines.
- Whether the pre-condition, test steps to execute and expected results are clear and concise.

Only output "PASS" if all criteria are met and you have no further suggestions for improvements.
Output your evaluation concisely in the following format.

<evaluation>
PASS, NEEDS_IMPROVEMENT, or FAIL
</evaluation>

<feedback>
What needs improvement and why.
</feedback>

Here are the inputs you need to perform task on
Requirement Context and Test cases: {Understanding}

"""

In [474]:
import os
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser

os.environ["GROQ_API_KEY"] = "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"

unpack={}

llm_worker = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
)

def _format_prompt_worker(template: str,  **kwargs) -> str:
    """Format a prompt template with variables."""
    try:
        return template.format(**kwargs)
    except KeyError as e:
        raise ValueError(f"Missing required prompt variable: {e}")
        

In [599]:
def generate_test_case(context: str, prompt: str, feedback: str = "") -> tuple[str, str]:
    
    full_prompt = f"{prompt}\n{feedback}\n" if feedback else f"{prompt}\n"
    
    print(f"*************** Generating Test case **********************")
            #for scenario in item['Test Scenarios']:
    worker_input = _format_prompt_worker(
                full_prompt,
                context_requirement=context,
                **unpack
    )
    #print(worker_input)
        
    worker_response = llm_worker.invoke(worker_input) 
    test_cases = extract_xml(worker_response.content, "test_cases")
    print(f"====== WORKER RESPONSE ======\n{test_cases}\n")
        #pprint(worker_response)

    return test_cases

In [600]:
TC_result = generate_test_case(sub_analysis, WORKER_PROMPT)

*************** Generating Test case **********************
====== WORKER RESPONSE ======

Test Case ID : 1
Objective : To verify the steering angle adjustment within the expected range (8° per 90° of steering wheel input) for correct wheel alignment.
Test Design Technique : Equivalence Partitioning
Pre-conditions : The vehicle is stationary, the steering wheel is at the center position, and the ECU is powered on.
Steps to Execute : 
1. Rotate the steering wheel by 90° in either direction.
2. Measure the resulting steering angle at the wheels.
Expected Results : 
1. The steering angle at the wheels should be within the range of 8° (±4°) of the expected value.

Test Case ID : 2
Objective : To validate the assistive torque application within the expected range (1Nm to 15Nm) based on driving speed and steering input.
Test Design Technique : Boundary Value Analysis
Pre-conditions : The vehicle is in motion, the ECU is powered on, and the steering wheel is at the center position.
Steps to E

In [601]:
def evaluate_test_case(prompt: str, content: str) -> tuple[str, str]:
    """Evaluate if a solution meets requirements."""
    full_prompt = f"{prompt}\nContent to evaluate: {content}"
    print("************** Evaluating Test Case ****************")
    eval_input = _format_prompt_worker(
                EVAL_PROMPT,
                Understanding=content,
                **unpack
    )
    #print(eval_input)
    eval_response = llm_worker.invoke(eval_input)
    evaluation = extract_xml(eval_response.content, "evaluation")
    feedback = extract_xml(eval_response.content, "feedback")
    #pprint(worker_response)
              
    print(f"\n=== Eval RESULT ===\n{evaluation}\n{feedback}")
    return (evaluation, feedback)
    

In [581]:
evaluation, feedback = evaluate_test_case(EVAL_PROMPT, TC_result)

************** Evaluating Test Case ****************

=== Eval RESULT ===

NEEDS_IMPROVEMENT


The test cases cover different aspects of the ECU's response to steering wheel angle input, including equivalence partitioning, boundary value analysis, and error guessing. However, there are a few areas that need improvement:

1. Test Case 1.1: The expected result for assist torque is too broad (1Nm to 15Nm). It would be better to specify a more precise range or a specific value based on the requirement.

2. Test Case 1.2: The boundary value of 180° is tested, but it would be more comprehensive to test the boundary values at both ends of the effective range (e.g., 0° and 180°).

3. Test Case 1.3: The expected result is not clear. What should happen when the input is beyond the effective range? Should the ECU ignore the input, or should it produce an error message? The expected result should be more specific.

Additionally, it would be beneficial to include test cases for invalid or out-of-ra

In [607]:
def feedback_loop(context: str, evaluator_prompt: str, generator_prompt: str) -> tuple[str, list[dict]]:
    """Keep generating and evaluating until requirements are met."""
    memory = []
    chain_of_thought = []
    
    test_cases = generate_test_case(context, generator_prompt)
    memory.append(test_cases)
    chain_of_thought.append(test_cases)
    while True:
        evaluation, feedback = evaluate_test_case(evaluator_prompt, test_cases)
        if evaluation.strip() == "PASS":
            return test_cases
            
        context = "\n".join([
            "Previous attempts:",
            *[f"- {m}" for m in memory],
            f"\nFeedback: {feedback}"
        ])
        
        test_cases = generate_test_case(context, generator_prompt)
        memory.append(test_cases)
        chain_of_thought.append(test_cases)

In [608]:
final_result = feedback_loop(sub_analysis, EVAL_PROMPT, WORKER_PROMPT)

*************** Generating Test case **********************
====== WORKER RESPONSE ======

Test Case ID : 1
Objective : To verify the steering angle adjustment within the expected range (8° per 90° of steering wheel input) for correct wheel alignment.
Test Design Technique : Equivalence Partitioning
Pre-conditions : The vehicle is stationary, the steering wheel is at the center position, and the ECU is powered on.
Steps to Execute : 
1. Rotate the steering wheel by 90° in either direction.
2. Measure the resulting steering angle at the wheels.
Expected Results : 
1. The steering angle at the wheels should be within the range of 8° (±4°) of the expected value.

Test Case ID : 2
Objective : To validate the assistive torque application within the expected range (1Nm to 15Nm) based on driving speed and steering input.
Test Design Technique : Boundary Value Analysis
Pre-conditions : The vehicle is in motion, the ECU is powered on, and the steering wheel is at the center position.
Steps to E

In [610]:
pprint(final_result)

('\n'
 'Test Case ID : 1\n'
 'Objective : To verify the steering angle adjustment within the expected '
 'range (8° per 90° of steering wheel input) for correct wheel alignment.\n'
 'Test Design Technique : Equivalence Partitioning\n'
 'Pre-conditions : The vehicle is stationary, the steering wheel is at the '
 'center position, and the ECU is powered on.\n'
 'Steps to Execute : \n'
 '1. Rotate the steering wheel by 90° in either direction.\n'
 '2. Measure the resulting steering angle at the wheels.\n'
 'Expected Results : \n'
 '1. The steering angle at the wheels should be within the range of 8° (±4°) '
 'of the expected value.\n'
 '\n'
 'Test Case ID : 2\n'
 'Objective : To validate the assistive torque application within the expected '
 'range (1Nm to 15Nm) based on driving speed and steering input.\n'
 'Test Design Technique : Boundary Value Analysis\n'
 'Pre-conditions : The vehicle is in motion, the ECU is powered on, and the '
 'steering wheel is at the center position.\n'
 'Ste

In [1]:
result = [{'Requirement Covered': 245103, 'Test Cases': "\n\nTest Case ID: 1\nObjective: To verify the ECU's response to steering wheel angles across the entire range (±180°) with high granularity.\nTest Design Technique: Equivalence Partitioning\nPre-conditions: The vehicle is in a stationary state, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the steering wheel angle to -180° and measure the ECU's assist torque response.\n2. Set the steering wheel angle to -170° and measure the ECU's assist torque response.\n3. Set the steering wheel angle to -160° and measure the ECU's assist torque response.\n...\n30. Set the steering wheel angle to 180° and measure the ECU's assist torque response.\nExpected Results:\nThe ECU's assist torque response should be accurate and within the specified range for each steering wheel angle.\n\nTest Case ID: 2\nObjective: To verify the ECU's response to steering wheel angles within the effective range (e.g., ±170°, ±160°, etc.).\nTest Design Technique: Boundary Value Analysis\nPre-conditions: The vehicle is in a stationary state, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the steering wheel angle to -170° and measure the ECU's assist torque response.\n2. Set the steering wheel angle to -160° and measure the ECU's assist torque response.\n3. Set the steering wheel angle to -150° and measure the ECU's assist torque response.\n...\n10. Set the steering wheel angle to 170° and measure the ECU's assist torque response.\nExpected Results:\nThe ECU's assist torque response should be accurate and within the specified range for each steering wheel angle.\n\nTest Case ID: 3\nObjective: To verify the ECU's assist torque response to different steering torque input values.\nTest Design Technique: Equivalence Partitioning\nPre-conditions: The vehicle is in a stationary state, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the steering torque input to 5Nm and measure the ECU's assist torque response.\n2. Set the steering torque input to 10Nm and measure the ECU's assist torque response.\n3. Set the steering torque input to 15Nm and measure the ECU's assist torque response.\n...\n10. Set the steering torque input to 50Nm and measure the ECU's assist torque response.\nExpected Results:\nThe ECU's assist torque response should be accurate and within the specified range for each steering torque input value.\n\nTest Case ID: 4\nObjective: To verify the ECU's assist torque and steering angle adjustments to different turning radius values.\nTest Design Technique: Equivalence Partitioning\nPre-conditions: The vehicle is in a stationary state, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the turning radius to 4.5 meters and measure the ECU's assist torque and steering angle adjustments.\n2. Set the turning radius to 5 meters and measure the ECU's assist torque and steering angle adjustments.\n3. Set the turning radius to 5.5 meters and measure the ECU's assist torque and steering angle adjustments.\n...\n10. Set the turning radius to 10 meters and measure the ECU's assist torque and steering angle adjustments.\nExpected Results:\nThe ECU's assist torque and steering angle adjustments should be accurate and within the specified range for each turning radius value.\n\nTest Case ID: 5\nObjective: To verify the ECU's assist torque response to different vehicle speed values.\nTest Design Technique: Equivalence Partitioning\nPre-conditions: The vehicle is in motion, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the vehicle speed to 50 km/h and measure the ECU's assist torque response.\n2. Set the vehicle speed to 75 km/h and measure the ECU's assist torque response.\n3. Set the vehicle speed to 100 km/h and measure the ECU's assist torque response.\n...\n10. Set the vehicle speed to 150 km/h and measure the ECU's assist torque response.\nExpected Results:\nThe ECU's assist torque response should be accurate and within the specified range for each vehicle speed value.\n\nTest Case ID: 6\nObjective: To verify the ECU's response to simultaneous changes in multiple input parameters (e.g., steering wheel angle and vehicle speed).\nTest Design Technique: State Transition Testing\nPre-conditions: The vehicle is in motion, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the steering wheel angle to -90° and the vehicle speed to 50 km/h, and measure the ECU's assist torque response.\n2. Set the steering wheel angle to 90° and the vehicle speed to 100 km/h, and measure the ECU's assist torque response.\n3. Set the steering wheel angle to -45° and the vehicle speed to 75 km/h, and measure the ECU's assist torque response.\n...\n10. Set the steering wheel angle to 135° and the vehicle speed to 125 km/h, and measure the ECU's assist torque response.\nExpected Results:\nThe ECU's assist torque response should be accurate and within the specified range for each combination of input parameters.\n\nTest Case ID: 7\nObjective: To verify the ECU's behavior during transition between different driving modes (e.g., from parking to high-speed driving).\nTest Design Technique: State Transition Testing\nPre-conditions: The vehicle is in a stationary state, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the vehicle to parking mode and measure the ECU's assist torque response.\n2. Transition the vehicle to high-speed driving mode and measure the ECU's assist torque response.\n3. Transition the vehicle to low-speed driving mode and measure the ECU's assist torque response.\n...\n5. Transition the vehicle to parking mode and measure the ECU's assist torque response.\nExpected Results:\nThe ECU's assist torque response should be accurate and within the specified range during each driving mode transition.\n\nTest Case ID: 8\nObjective: To verify the ECU's response to edge cases, such as maximum and minimum values for each input parameter.\nTest Design Technique: Boundary Value Analysis\nPre-conditions: The vehicle is in a stationary state, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the steering wheel angle to -180° and measure the ECU's assist torque response.\n2. Set the steering wheel angle to 180° and measure the ECU's assist torque response.\n3. Set the steering torque input to 0Nm and measure the ECU's assist torque response.\n...\n10. Set the vehicle speed to 0 km/h and measure the ECU's assist torque response.\nExpected Results:\nThe ECU's assist torque response should be accurate and within the specified range for each edge case.\n\n"}, {'Requirement Covered': 245105, 'Test Cases': '\n\nTest Case ID: 1\nObjective: To verify that the Steering ECU correctly adjusts the wheel steering angle based on the steering ratio for different input ranges of steering wheel angle.\nTest Design Technique: Equivalence Partitioning\nPre-conditions: The Steering ECU is initialized with a valid steering ratio. The steering wheel angle input is within the valid range.\nSteps to Execute:\n1. Set the steering ratio to a valid value (e.g., 1:1).\n2. Input a series of steering wheel angles (e.g., 0°, 30°, 60°, 90°, 120°, 150°, 180°) and measure the corresponding wheel steering angle at the wheels.\n3. Repeat step 2 for different steering ratios (e.g., 1:2, 1:3).\nExpected Results:\n* The wheel steering angle at the wheels is correctly adjusted based on the steering ratio and input steering wheel angle.\n* The wheel steering angle is within the expected range for each input steering wheel angle.\n\nTest Case ID: 2\nObjective: To verify that the Steering ECU correctly applies assistive torque based on the driving scenario.\nTest Design Technique: Scenario-Based Testing\nPre-conditions: The Steering ECU is initialized with a valid driving scenario. The assistive torque is enabled.\nSteps to Execute:\n1. Set the driving scenario to a specific value (e.g., city driving, highway driving).\n2. Input a series of steering wheel angles (e.g., 0°, 30°, 60°) and measure the corresponding assistive torque applied.\n3. Repeat step 2 for different driving scenarios (e.g., high-speed driving, low-speed driving).\nExpected Results:\n* The assistive torque is correctly applied based on the driving scenario and input steering wheel angle.\n* The assistive torque is within the expected range for each driving scenario.\n\nTest Case ID: 3\nObjective: To verify that the Steering ECU correctly responds to extreme steering wheel angles.\nTest Design Technique: Boundary Value Analysis\nPre-conditions: The Steering ECU is initialized with a valid steering ratio. The steering wheel angle input is within the extreme range.\nSteps to Execute:\n1. Input a series of extreme steering wheel angles (e.g., -450°, -300°, -150°, 150°, 300°, 450°) and measure the corresponding wheel steering angle at the wheels.\n2. Repeat step 1 for different steering ratios (e.g., 1:1, 1:2).\nExpected Results:\n* The wheel steering angle at the wheels is correctly adjusted based on the steering ratio and input extreme steering wheel angle.\n* The wheel steering angle is within the expected range for each input extreme steering wheel angle.\n\nTest Case ID: 4\nObjective: To verify that the Steering ECU correctly responds to rapid changes in steering wheel angle.\nTest Design Technique: State Transition Testing\nPre-conditions: The Steering ECU is initialized with a valid steering ratio. The steering wheel angle input is within the valid range.\nSteps to Execute:\n1. Input a rapid series of steering wheel angles (e.g., 0°, 30°, 60°, 90°, 120°, 150°, 180°) and measure the corresponding wheel steering angle at the wheels.\n2. Repeat step 1 for different steering ratios (e.g., 1:1, 1:2).\nExpected Results:\n* The wheel steering angle at the wheels is correctly adjusted based on the steering ratio and input rapid steering wheel angle changes.\n* The wheel steering angle is within the expected range for each input rapid steering wheel angle change.\n\nTest Case ID: 5\nObjective: To verify that the Steering ECU correctly responds to simultaneous changes in steering wheel angle and driving scenario.\nTest Design Technique: Use Case Testing\nPre-conditions: The Steering ECU is initialized with a valid steering ratio and driving scenario. The assistive torque is enabled.\nSteps to Execute:\n1. Input a series of simultaneous changes in steering wheel angle (e.g., 0°, 30°, 60°) and driving scenario (e.g., city driving, highway driving) and measure the corresponding wheel steering angle at the wheels and assistive torque applied.\n2. Repeat step 1 for different steering ratios (e.g., 1:1, 1:2) and driving scenarios (e.g., high-speed driving, low-speed driving).\nExpected Results:\n* The wheel steering angle at the wheels is correctly adjusted based on the steering ratio, input steering wheel angle, and driving scenario.\n* The assistive torque is correctly applied based on the driving scenario and input steering wheel angle.\n* The wheel steering angle and assistive torque are within the expected range for each input simultaneous change.\n\nTest Case ID: 6\nObjective: To verify that the Steering ECU correctly responds to invalid or out-of-range input values.\nTest Design Technique: Error Guessing\nPre-conditions: The Steering ECU is initialized with a valid steering ratio. The steering wheel angle input is invalid or out-of-range.\nSteps to Execute:\n1. Input a series of invalid or out-of-range steering wheel angles (e.g., -500°, 500°, null) and measure the corresponding wheel steering angle at the wheels.\n2. Repeat step 1 for different steering ratios (e.g., 1:1, 1:2).\nExpected Results:\n* The Steering ECU correctly handles invalid or out-of-range input values and does not crash or produce unexpected results.\n* The wheel steering angle at the wheels is not affected by invalid or out-of-range input values.\n'}]

In [21]:
res = result[0].get("Test Cases", "").strip()

In [22]:
res

"Test Case ID: 1\nObjective: To verify the ECU's response to steering wheel angles across the entire range (±180°) with high granularity.\nTest Design Technique: Equivalence Partitioning\nPre-conditions: The vehicle is in a stationary state, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the steering wheel angle to -180° and measure the ECU's assist torque response.\n2. Set the steering wheel angle to -170° and measure the ECU's assist torque response.\n3. Set the steering wheel angle to -160° and measure the ECU's assist torque response.\n...\n30. Set the steering wheel angle to 180° and measure the ECU's assist torque response.\nExpected Results:\nThe ECU's assist torque response should be accurate and within the specified range for each steering wheel angle.\n\nTest Case ID: 2\nObjective: To verify the ECU's response to steering wheel angles within the effective range (e.g., ±170°, ±160°, etc.).\nTest Design Technique: Boundary Value Analysis\nPre-conditions: 

In [23]:
buffer_testcase = res.split("Test Case ID : ")
buffer_testcase

["Test Case ID: 1\nObjective: To verify the ECU's response to steering wheel angles across the entire range (±180°) with high granularity.\nTest Design Technique: Equivalence Partitioning\nPre-conditions: The vehicle is in a stationary state, and the ECU is in a default configuration.\nSteps to Execute:\n1. Set the steering wheel angle to -180° and measure the ECU's assist torque response.\n2. Set the steering wheel angle to -170° and measure the ECU's assist torque response.\n3. Set the steering wheel angle to -160° and measure the ECU's assist torque response.\n...\n30. Set the steering wheel angle to 180° and measure the ECU's assist torque response.\nExpected Results:\nThe ECU's assist torque response should be accurate and within the specified range for each steering wheel angle.\n\nTest Case ID: 2\nObjective: To verify the ECU's response to steering wheel angles within the effective range (e.g., ±170°, ±160°, etc.).\nTest Design Technique: Boundary Value Analysis\nPre-conditions:

In [30]:
 for test_case in buffer_testcase:
     lines = [line.strip() for line in test_case.split("\n") if line.strip()]
     test_case_id = lines[0]  # Test Case ID is the first line
     objective = lines[1].split(":", 1)[1].strip() if ":" in lines[1] else "N/A"
     technique = lines[2].split(":", 1)[1].strip() if ":" in lines[2] else "N/A"
     pre_conditions = lines[3].split(":", 1)[1].strip() if ":" in lines[3] else "N/A"
     steps_start = next((i for i, line in enumerate(lines) if "Steps to Execute" in line), 4)
     results_start = next((i for i, line in enumerate(lines) if "Expected Results" in line), len(lines))
     steps = "\n".join(lines[steps_start + 1: results_start]).strip()
     expected_results = "\n".join(lines[results_start + 1:]).strip()

     # print(f"{test_case_id}\n")
     # print(f"{objective}\n")
     print(expected_results)

The ECU's assist torque response should be accurate and within the specified range for each steering wheel angle.
Test Case ID: 2
Objective: To verify the ECU's response to steering wheel angles within the effective range (e.g., ±170°, ±160°, etc.).
Test Design Technique: Boundary Value Analysis
Pre-conditions: The vehicle is in a stationary state, and the ECU is in a default configuration.
Steps to Execute:
1. Set the steering wheel angle to -170° and measure the ECU's assist torque response.
2. Set the steering wheel angle to -160° and measure the ECU's assist torque response.
3. Set the steering wheel angle to -150° and measure the ECU's assist torque response.
...
10. Set the steering wheel angle to 170° and measure the ECU's assist torque response.
Expected Results:
The ECU's assist torque response should be accurate and within the specified range for each steering wheel angle.
Test Case ID: 3
Objective: To verify the ECU's assist torque response to different steering torque input